In [ ]:
import torch
print(torch.cuda.is_available())  # 应该返回 True
print(torch.cuda.device_count())  # 返回 GPU 数量
print(torch.cuda.get_device_name(0))  # 返回 GPU 名称
print(torch.cuda.current_device())  # 返回当前 GPU 的索引号
print(torch.__version__)  # 查看 PyTorch 版本
print(torch.version.cuda)  # 查看 PyTorch 编译时的 CUDA 版本
print(torch.backends.cudnn.version())  # 检查 cuDNN 版本


In [ ]:
import torch
#   检查 CUDA 是否可用
device = torch.device("cuda")

# 让 GPU 计算 5000 x 5000 的矩阵乘法
a = torch.rand(5000, 5000, device=device)
b = torch.rand(5000, 5000, device=device)
c = torch.matmul(a, b)

print(torch.cuda.get_device_name(0))

# 保持进程运行一段时间，观察 nvidia-smi
import time
time.sleep(30)  # 等待 30 秒
print("done")

In [4]:
import torch
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

a = torch.randn(10000, 10000, device=device)
b = torch.randn(10000, 10000, device=device)

start = time.time()
c = torch.matmul(a, b)
torch.cuda.synchronize()  # 确保 GPU 计算完成
end = time.time()

# 计算当前 GPU 的显存占用情况
print(torch.cuda.memory_allocated(0) / 1024**2, "MB 已分配")
print(torch.cuda.memory_reserved(0) / 1024**2, "MB 已保留")
print(torch.cuda.max_memory_allocated(0) / 1024**2, "MB 最大分配")

print(f"GPU 耗时: {end - start:.5f} 秒 ", c.device)

device = torch.device("cpu")

a = torch.randn(10000, 10000, device=device)
b = torch.randn(10000, 10000, device=device)

start = time.time()
c = torch.matmul(a, b)
end = time.time()

# 计算当前 GPU 的显存占用情况
print(torch.cuda.memory_allocated(0) / 1024**2, "MB 已分配")
print(torch.cuda.memory_reserved(0) / 1024**2, "MB 已保留")
print(torch.cuda.max_memory_allocated(0) / 1024**2, "MB 最大分配")

print(f"CPU 耗时: {end - start:.5f} 秒", c.device)


1154.125 MB 已分配
1166.0 MB 已保留
1154.125 MB 最大分配
GPU 耗时: 0.82705 秒  cuda:0
8.125 MB 已分配
1166.0 MB 已保留
1154.125 MB 最大分配
CPU 耗时: 4.20682 秒 cpu


假设输入的是两个不同的特征张量（模拟查询、键值对的情况），展示如何通过CrossAttention模块进行信息融合。

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import math

# 定义CrossAttention类
class CrossAttention(nn.Module):
    def __init__(
            self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0.,
            proj_drop=0., window_size=None, attn_head_dim=None):
        super().__init__()
        self.num_heads = num_heads      # 头的数量
        # // 双斜杠是整数除法运算符，返回不大于结果的一个最大的整数
        # /  单斜杠是浮点数除法运算符，返回浮点数
        head_dim = dim // num_heads     # 每个头的维度 = 总维度 / 头的数量
        if attn_head_dim is not None:
            head_dim = attn_head_dim
        all_head_dim = head_dim * self.num_heads    # 所有头的维度
        self.scale = qk_scale or head_dim ** -0.5   # 缩放因子

        self.q = nn.Linear(dim, all_head_dim, bias=False)   # bias=False表示不使用偏置  
        self.k = nn.Linear(dim, all_head_dim, bias=False)
        self.v = nn.Linear(dim, all_head_dim, bias=False)

        if qkv_bias:    # 如果使用偏置 偏置初始化为0
            self.q_bias = nn.Parameter(torch.zeros(all_head_dim))   
            self.k_bias = nn.Parameter(torch.zeros(all_head_dim))
            self.v_bias = nn.Parameter(torch.zeros(all_head_dim))
        else:
            self.q_bias = None
            self.k_bias = None
            self.v_bias = None

        # Dropout 是一种正则化技术，通过在训练过程中随机丢弃一部分神经元来防止过拟合。
        # 它可以提高模型的泛化能力。
        self.attn_drop = nn.Dropout(attn_drop)      #   dropout层 
        self.proj = nn.Linear(all_head_dim, dim)    #   线性映射层
        self.proj_drop = nn.Dropout(proj_drop)      #   dropout层

    #   前向传播函数
    def forward(self, x, bool_masked_pos=None, k=None, v=None):
        # B: batch_size, N: 序列长度, C: 特征维度
        B, N, C = x.shape   
        N_k = k.shape[1]
        N_v = v.shape[1]

        q_bias, k_bias, v_bias = None, None, None
        if self.q_bias is not None:
            q_bias = self.q_bias
            # 设定一个和v_bias一样大小的全0张量  k_bias
            # requires_grad=False表示不需要计算梯度，训练中不需要更新
            k_bias = torch.zeros_like(self.v_bias, requires_grad=False) 
            v_bias = self.v_bias

        #   F.linear(input, weight, bias=None) → Tensor
        #   input: 输入张量
        #   weight: 权重张量
        #   bias: 偏置张量
        q = F.linear(input=x, weight=self.q.weight, bias=q_bias)
        q = q.reshape(B, N, 1, self.num_heads, -1).permute(2, 0, 3, 1, 4).squeeze(0)    # (B, N_head, N_q, dim)

        k = F.linear(input=k, weight=self.k.weight, bias=k_bias)
        k = k.reshape(B, N_k, 1, self.num_heads, -1).permute(2, 0, 3, 1, 4).squeeze(0)

        v = F.linear(input=v, weight=self.v.weight, bias=v_bias)
        v = v.reshape(B, N_v, 1, self.num_heads, -1).permute(2, 0, 3, 1, 4).squeeze(0)

        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))      # (B, N_head, N_q, N_k)

        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, -1)
        x = self.proj(x)
        x = self.proj_drop(x)

        return x


# 设置相关的维度参数和输入张量示例
batch_size = 80  # 批次大小
dim = 64  # 特征维度
num_heads = 8  # 头的数量
seq_len_query = 20  # 查询序列长度
seq_len_key_value = 10  # 键值对序列长度

# 随机生成输入张量，模拟查询、键、值
query = torch.rand(batch_size, seq_len_query, dim)
key = torch.rand(batch_size, seq_len_key_value, dim)
value = torch.rand(batch_size, seq_len_key_value, dim)

# 实例化CrossAttention模块
cross_attention_module = CrossAttention(dim=dim, num_heads=num_heads)

# 进行前向传播计算
output = cross_attention_module(query, k=key, v=value)

print("输入张量的形状:", query.shape)
print("键张量的形状:", key.shape)
print("值张量的形状:", value.shape)
print("输出结果的形状:", output.shape)

首先定义了CrossAttention类，包括线性变换、维度调整、注意力权重计算以及最终的信息融合和输出投影等操作。

然后设置了一些示例参数，比如批次大小、特征维度、头的数量以及查询和键值对的序列长度等，并随机生成了对应的输入张量（模拟查询、键、值）。

接着实例化了CrossAttention模块，并使用生成的输入张量进行前向传播计算，最后打印出输出结果的形状。